In [30]:
import tensorflow as tf
from IPython.display import display
import pandas as pd
import seaborn as sns
import os
import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import BatchNormalization, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau
from sklearn.utils import class_weight
%matplotlib inline

In [5]:
from helper import *

In [7]:
image_ext = '.jpeg'
data_dir = 'dataset/'
test_dir = data_dir+'test/'
train_dir = data_dir+'train/'
val_dir = data_dir+'val/'
normals = 'NORMAL/'
infected = 'PNEUMONIA/'
training_normal_nos = len(glob.glob1(train_dir+normals,'*'+image_ext))
training_infected_nos = len(glob.glob1(train_dir+infected,'*'+image_ext))
training_nos = training_infected_nos+training_normal_nos
testing_normal_nos = len(glob.glob1(test_dir+normals,'*'+image_ext))
testing_infected_nos = len(glob.glob1(test_dir+infected,'*'+image_ext))
testing_nos = testing_infected_nos+testing_normal_nos
validation_normal_nos = len(glob.glob1(val_dir+normals,'*'+image_ext))
validation_infected_nos = len(glob.glob1(val_dir+infected,'*'+image_ext))
validation_nos = validation_infected_nos+validation_normal_nos

print('Total number of training images: ', training_nos)
print('Total number of testing images: ', testing_nos)
print('Total number of validation images: ', validation_nos)

Total number of training images:  5216
Total number of testing images:  624
Total number of validation images:  16


In [41]:
rescale_factor = 1./255
target_size = (150,150,3)
class_mode = 'categorical'
training_batch_size = int(training_nos/40)

train_data_generator = create_image_data_generator(target_size=target_size, 
                                                   rescale=rescale_factor, 
                                                   path=train_dir, 
                                                   batch_size=training_batch_size)

test_data_generator = create_image_data_generator(target_size=target_size,
                                                 rescale=rescale_factor,
                                                 path=test_dir,
                                                 batch_size=testing_nos)

val_data_generator = create_image_data_generator(target_size=target_size,
                                                 rescale=rescale_factor,
                                                 path=val_dir,
                                                 batch_size=validation_nos)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [40]:
model_dir = 'models/'
log_dir = 'logs/'

In [42]:
inception_model = InceptionV3(weights='imagenet', include_top=False, input_shape = target_size)
x = inception_model.output
x = BatchNormalization()(x)
predictions = Dense(2,activation='softmax')(x)
for layer in inception_model.layers:
    layer.trainable = False
model = Model(inputs = inception_model.input, outputs=predictions)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [43]:
tf_reset_callbacks()
tf_reset_graph()
model_file = model_dir + "{epoch:02d}-val_acc-{val_acc:.2f}-val_loss-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(model_file,
                            monitor='val_acc',
                            save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss',
                              patience=5,
                              verbose=1,
                              restore_best_weights=True)
tensorboard = TensorBoard(log_dir=log_dir,
                         batch_size=training_batch_size,
                         update_freq='batch')
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                             patience=5,
                             cooldown=2,
                             min_lr=1e-8,
                             verbose=1)
model_callbacks = [checkpoint, early_stopping, tensorboard, reduce_lr]

In [44]:
steps_per_epoch = np.ceil(training_nos/training_batch_size)
validation_steps = 1

optimizer = Adam()
model.compile(optimizer=optimizer, loss = 'binary_crossentropy', metrics=['accuracy'])
history = model.fit_generator(train_data_generator,
                             steps_per_epoch=steps_per_epoch,
                             epochs=100,
                             verbose=2,
                             validation_data=val_data_generator,
                             validation_steps=validation_steps,
                             class_weight=[1.9448173 , 0.67303226],
                             callbacks=model_callbacks)

ValueError: Tensor("dense/truediv:0", shape=(?, 3, 3, 2), dtype=float32) must be from the same graph as Tensor("dense_target:0", shape=(?, ?, ?, ?), dtype=float32).